### PRIMER GAP AL ALZA
El primer gap al alza es la primera senal de cambio de tendencia y el anuncio del fin de una caida,

Los precios caen de forma inportanten y nos encontramos cerca de Piso Fuerte osea PM de 100 o de 200 en diari

Luego se  presenta en hora un Salto, la primera vela debe ser verde 9:30 a 10am ,  en esta vela debemos trazar un pizo y durante todo el dia los 
precios no pueden romper dicho piso
Es un requisito que una de las velas verdes debe tener volumen, esta vela debe ser alcista con cuerpos superiores a las c
las
Cuando el Primer gap al alza se presenta, podemos decir que el mercado retoma su tendencia al alza
y que la caida sera recuperada en un tiempo corto, los precios suben de 3 dias a 1 semana, entre 10 y 12 
untos
Para comprar el Primer gap al alza se debe esperar hasta las 3:57pm
A partir de que se da el Gap es importante que la vela de apertura de los proximos dias sea verde, y que
respete el piso del gap de
cada dia
El primer Gap al Alza Se presenta cada 3 o 4 meses en la tendencia al alza, normalmente despues de caid
 brutales
Se compra para una semana, los precios suben de 2 a 3 dias maximo en una semana

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import yfinance as yf
import talib as ta
import matplotlib.pyplot as plt
import math
from datetime import date, timedelta
from scipy import stats
from scipy.signal import argrelextrema
from bokeh.plotting import figure, show, column
from sklearn.linear_model import LinearRegression

In [3]:
df_d = pd.read_csv(r"D:\traderxpro\data\dataxd.txt", sep="\t")
tickers = [
    'SPY',
    'META',
    'AAPL'
]

df_h = pd.read_csv(r"D:\traderxpro\data\dataxh.txt", sep="\t")
tickers = [
    'SPY',
    'META',
    'AAPL'
]

#    'SPY',
#    'META',
#    'AAPL',
#    'AMZN',
#    'NFLX',
#    'MRNA',
#    'TSLA',
#    'TNA',
#    'GLD',
#    'SLV',
#    'USO',
#    'BAC',
#    'CVX',
#    'XOM',
#    'QQQ'

In [8]:
df_d=df_d.rename(columns={'datetime': 'fecha'})
df_d['fecha'] = pd.to_datetime(df_d['fecha'])
df_dia = pd.DataFrame()
for ticker in tickers:
    company = df_d.query("companyName==@ticker").copy()
    company.sort_values(by=['fecha'])
    #company = yf.download(ticker, start = ini2_str, end = today_str, interval='60m')
    #company.columns = [company.columns[0][0], company.columns[1][0], company.columns[2][0], company.columns[3][0], company.columns[4][0]]
    #company.rename(columns={'Datetime':'Gmt time'}, inplace = True)
    #company['datetime'] = pd.to_datetime (company.index)
    #company['companyName'] = ticker
    company['SMA20'] = company['Close'].rolling(20).mean()
    company.dropna(inplace=False)
    company['SMA40'] = company['Close'].rolling(40).mean()
    company.dropna(inplace=False)
    company['SMA100'] = company['Close'].rolling(100).mean()
    company.dropna(inplace=False)
    company['SMA200'] = company['Close'].rolling(200).mean()
    company.dropna(inplace=False)
    df_dia = pd.concat([df_dia, company],ignore_index=True)

In [10]:
df_h['datetime'] = pd.to_datetime(df_h['datetime'])
df_h["fecha"] = pd.to_datetime(df_h["datetime"].dt.date)
df_hora = pd.DataFrame()
for ticker in tickers:
    company = df_h.query("companyName==@ticker").copy()
    company.sort_values(by=['datetime'])
    #company = yf.download(ticker, start = ini2_str, end = today_str, interval='60m')
    #company.columns = [company.columns[0][0], company.columns[1][0], company.columns[2][0], company.columns[3][0], company.columns[4][0]]
    #company.rename(columns={'Datetime':'Gmt time'}, inplace = True)
    #company['datetime'] = pd.to_datetime (company.index)
    #company['companyName'] = ticker
    company['SMA20'] = company['Close'].rolling(20).mean()
    company.dropna(inplace=False)
    company['SMA40'] = company['Close'].rolling(40).mean()
    company.dropna(inplace=False)
    company['SMA100'] = company['Close'].rolling(100).mean()
    company.dropna(inplace=False)
    company['SMA200'] = company['Close'].rolling(200).mean()
    company.dropna(inplace=False)
    df_hora = pd.concat([df_hora, company],ignore_index=True)

### Detect Pivots/Fractals

In [13]:
ord=5
#for ticker in tickers:
#print("====", ticker)
#DIA
max_idx1 = argrelextrema(df_dia['Close'].values, np.greater, order=ord)[0]
min_idx1 = argrelextrema(df_dia['Close'].values, np.less, order=ord)[0]
# Aplicar el cálculo solo a los índices en la lista
df_dia.loc[max_idx1, 'pivotHigh'] = df_dia['High']+1e-3
df_dia.loc[min_idx1, 'pivotLow'] = df_dia['Low']-(1e-3)
df_dia.loc[max_idx1, 'isPivot'] = 1
df_dia.loc[min_idx1, 'isPivot'] = 2

#HORA
max_idx2 = argrelextrema(df_hora['Close'].values, np.greater, order=ord)[0]
min_idx2 = argrelextrema(df_hora['Close'].values, np.less, order=ord)[0]
# Aplicar el cálculo solo a los índices en la lista
df_hora.loc[max_idx2, 'pivotHigh'] = df_hora['High']+1e-3
df_hora.loc[min_idx2, 'pivotLow'] = df_hora['Low']-(1e-3)
df_hora.loc[max_idx2, 'isPivot'] = 1
df_hora.loc[min_idx2, 'isPivot'] = 2

In [15]:
#CODIGO DE REVISION DIA
#dfpl = df_hora.iloc[500:970]
dfpl = df_dia.iloc[0:150]

p = figure(width=2500, height=500,
        title="DIA",
        background_fill_color="#efefef",
        tooltips=[("Index", "@index"), ("Open", "@Open"), ("High","@High"), ("Low","@Low"), ("Close","@Close")]
        )
p.xaxis.major_label_orientation = 0.8 # radians
p.x_range.range_padding = 0.05
#p.xaxis.axis_line_join = "bevel" # radians
p.xaxis.axis_line_width = 2

p.segment("index", "High", "index","Low",  color="black", line_width=1, source=dfpl)

inc = dfpl.query("Close>Open")
dec = dfpl.query("Open>Close")

p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="red",
    line_color="red",    
    source=dec   
)


p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="green",
    line_color="green", 
    source=inc   
)

p.scatter(x="index", y="pivotHigh", marker="circle", size=10,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista", source=dfpl[(dfpl.isPivot==1)])

p.scatter(x="index", y="pivotLow", marker="circle", size=10,
           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista", source=dfpl[(dfpl.isPivot==2)])

show(p)

In [17]:
#CODIGO DE REVISION HORA
#dfpl = df_hora.iloc[500:970]
dfpl = df_hora.iloc[0:150]

p = figure(width=2500, height=500,
        title="HORA",
        background_fill_color="#efefef",
        tooltips=[("Index", "@index"), ("Open", "@Open"), ("High","@High"), ("Low","@Low"), ("Close","@Close")]
        )
p.xaxis.major_label_orientation = 0.8 # radians
p.x_range.range_padding = 0.05
#p.xaxis.axis_line_join = "bevel" # radians
p.xaxis.axis_line_width = 2

p.segment("index", "High", "index","Low",  color="black", line_width=1, source=dfpl)

inc = dfpl.query("Close>Open")
dec = dfpl.query("Open>Close")

p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="red",
    line_color="red",    
    source=dec   
)


p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="green",
    line_color="green", 
    source=inc   
)

p.scatter(x="index", y="pivotHigh", marker="circle", size=10,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista", source=dfpl[(dfpl.isPivot==1)])

p.scatter(x="index", y="pivotLow", marker="circle", size=10,
           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista", source=dfpl[(dfpl.isPivot==2)])

show(p)

In [19]:
## cerca a PM100 o PM200 en diario
def zonabarata(df_dia):
    if ((df_dia['isPivot']==2) & ((df_dia['Low']<=df_dia['SMA100']) | (df_dia['Low']<=df_dia['SMA200']))):
    #if ((df_dia['isPivot']==2) & (( df_dia['SMA100']-df_dia['Low']<=0.5 ) | ( df_dia['SMA200']-df_dia['Low']<=0.5 ))):    
        return 1
    else:
        return 0
df_dia['zbarata'] = df_dia.apply(zonabarata, axis=1)

In [21]:
df_hora_final = pd.merge(df_hora, df_dia[['fecha','companyName', 'zbarata']], on = ['companyName','fecha'], how='left')

In [23]:
#df_hora_final["Lownext"] = df_hora_final["Low"].shift(-1)
#df_hora_final["Lownext2"] = df_hora_final["Low"].shift(1)

In [25]:
#df_hora_final[df_hora_final['zbarata']==1]
def pga(df_hora_final):
    if ((df_hora_final['zbarata']==1) & (df_hora_final['isPivot']==2)):
        return 1
    else:
        return 0
df_hora_final['pga'] = df_hora_final.apply(pga, axis=1)

In [27]:
df_hora_final['Vela_Verde'] = df_hora_final['Close'] > df_hora_final['Open']

In [29]:
#Posibles casos GAP
df_casos = pd.DataFrame()
caso = 0
ticker2 = ""
window=5
backeval=75
backevalTrend=0
dfpl = pd.DataFrame
trendH = []
trendL = []
cant = 0
piso = 0
for candle, row in df_hora_final.iterrows():    
   #if (candle==118):
    if((df_hora_final['pga'][candle]==1)): #posibles casos 
        
        ticker = df_hora_final['companyName'][candle]
        #df_ticker=df_hora_final[(df_hora_final['companyName']==ticker) & (df_hora_final.index>=candle)].copy()
        df_ticker = df_hora_final[(df_hora_final.companyName==ticker)].loc[candle-backeval:candle+backeval].copy()            
        fechaini = df_hora_final['fecha'][candle]
        print("posible caso:",candle, df_hora_final['companyName'][candle], ", fecha:", fechaini)
        indice_ini =   candle + 1        
        if ((df_ticker.loc[indice_ini,"Low"]>df_ticker.loc[candle,"Low"]) & (df_hora_final['Vela_Verde'][indice_ini]==True)):
            print("PASO")
            fechaini = df_hora_final['fecha'][indice_ini]
            print(fechaini)
            #Si es verdadero seguir revisando las demas filas
            df_ticker.loc[indice_ini,'indicador'] = 1
            piso = df_hora_final.loc[indice_ini,"Low"]
            print(piso)
            #revisar ultima vela del dia
            #Generar fechas día a día entre el mínimo y el máximo
            rango = pd.date_range(start=fechaini, end=df_ticker['fecha'].max(), freq='D')

            #Recorrer cada día
            for fecha in rango:                    
                if (df_ticker.query("fecha==@fecha").shape[0]>0):
                    print(f"Procesando: {fecha.date()}")
                   
                    if (fechaini==fecha):
                        indice_i=indice_ini
                    else:
                        indice_i = df_ticker.query("fecha==@fecha").index[0] #inicio

                    indice_f = df_ticker.query("fecha==@fecha").index[-1] #fin
                    monto_ini = df_ticker['Close'][indice_i]
                    monto_fin = df_ticker['Close'][indice_f]

                    monto_ini2 = df_ticker['Low'][indice_i]
                    monto_fin2 = df_ticker['Low'][indice_f]  
                    
                    print("indice_i:",indice_i,",monto_ini:",monto_ini,",indice_f:",indice_f,",monto_fin:",monto_fin)

                    #Validacion vela inicial mayor a vela
                    ayer = fecha - timedelta(days=1)
                    if ((df_ticker.query("fecha<@fecha").shape[0]>0) & (fecha>fechaini)):
                        indice_ayer = df_ticker.query("fecha<@fecha").index[-1] #fin  
                        monto_finayer = df_ticker['Close'][indice_ayer]
                        monto_finayer2 = df_ticker['Low'][indice_ayer]
                        if ((monto_finayer>monto_ini) & (monto_finayer2>monto_ini2)):
                            df_ticker.loc[indice_ayer,'indicador'] = 2
                            break
                    
                    if ((monto_ini > monto_fin) & (monto_ini2 > monto_fin2) & (fecha>fechaini)):
                        df_ticker.loc[indice_f,'indicador'] = 2
                        break
                        
            #INSERT CASO
            df_ticker.loc[candle,'ind_posicion'] = 0
            caso=caso+1                        
            df_ticker["caso"] = caso
            #print(candle, df_ticker['companyName'][candle])
            if len(df_casos)<1:
                df_casos = df_ticker
            else:
                df_casos = pd.concat([df_casos, df_ticker],ignore_index=False)

posible caso: 595 SPY , fecha: 2023-10-03 00:00:00
PASO
2023-10-03 00:00:00
420.4299926757813
Procesando: 2023-10-03
indice_i: 596 ,monto_ini: 420.6199951171875 ,indice_f: 597 ,monto_fin: 421.6700134277344
Procesando: 2023-10-04
indice_i: 598 ,monto_ini: 420.6195983886719 ,indice_f: 604 ,monto_fin: 424.7000122070313
Procesando: 2023-10-05
indice_i: 605 ,monto_ini: 423.4249877929688 ,indice_f: 611 ,monto_fin: 424.4599914550781
posible caso: 721 SPY , fecha: 2023-10-27 00:00:00
posible caso: 2053 SPY , fecha: 2024-08-05 00:00:00
PASO
2024-08-05 00:00:00
514.6199951171875
Procesando: 2024-08-05
indice_i: 2054 ,monto_ini: 519.6799926757812 ,indice_f: 2059 ,monto_fin: 517.4099731445312
Procesando: 2024-08-06
indice_i: 2060 ,monto_ini: 522.9299926757812 ,indice_f: 2066 ,monto_fin: 522.1900024414062
Procesando: 2024-08-07
indice_i: 2067 ,monto_ini: 528.75 ,indice_f: 2073 ,monto_fin: 518.6500244140625
posible caso: 3105 SPY , fecha: 2025-03-13 00:00:00
posible caso: 3234 SPY , fecha: 2025-04-0

In [31]:
#descarga de informacion, estrategia: RUPTURA DEL CANAL BAJISTA
import os
path = r'D:\traderxpro\data\pga.txt'
# check whether the file exists
if os.path.exists(path):
    # delete the file
    os.remove(path)
else:
    # if the file does not exist.
    print("File does not exists. File needs to be created.")

#export DataFrame to text file
with open(path, 'a') as f:
    #df_string = appl_hor3.to_string(header=True, index=False, sep ='\t')
    df_casos.to_csv(path, header=True, index=None, sep='\t', mode='w')
    #f.write(df_string)

BACKTESTING
Usando paquete backtesting.py

In [34]:
from backtesting import Backtest, Strategy

C:\Users\carlo\anaconda3\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [36]:
#df_casos[df_casos["indicador"]>0]
df_casos2 = df_casos.query("indicador>0").copy()

In [40]:
# =============== BACKTESTING ================= #
# obteniendo dataframe backtesting
df_backTesting = pd.merge(df_hora, df_casos2[['datetime','companyName', 'indicador','caso']], on = ['companyName','datetime'], how='left')
# datos de trade se asigna una vela anterior debido a que backtesting.py toma entra como la siguiente vela
df_backTesting['indicador'] = df_backTesting['indicador'].shift(-1)
#modificamos indice con el compo datetime
df_backTesting.set_index('datetime', inplace=True)

In [42]:
#Strategia
class strategyPGA(Strategy):
    def init(self):
        self.indpga = self.I(lambda x: x, self.data.indicador) #Indica la señal entrada 1, salida 2
    def next(self):
        #print(self.data)
        if self.indpga[-1] == 1:
            self.buy()  #COMPRA
        elif self.indpga[-1] == 2 and self.position:
            self.position.close() #CIERRE

In [44]:
# Por cada TIcket
my_stats = {}
estadisticas = pd.DataFrame()          # lista para ticker
tradesprev = pd.DataFrame()     # lista para cada caso por ticker
for ticker in df_backTesting['companyName'].unique():
    ticker_data = df_backTesting[df_backTesting['companyName'] == ticker].copy() #Filtro por cada ticker
    #ticker_data=ticker_data.sort_values(by='caso')
    #print("\n" + "="*50 + "\n")
    bt = Backtest(ticker_data, strategyPGA, cash=10_000)
    #stats = bt.run()
    #my_stats[f"stats_BreakOUT_{ticker}"] = bt.run()
    my_stats = bt.run()
    listTrades = my_stats['_trades']
    tradesdf = pd.DataFrame(listTrades) #trades
    tradesdf['Ticker'] = ticker
    my_stast_filter=pd.DataFrame([my_stats])
    if tradesprev.shape[0]==0:
        tradesprev = tradesdf.copy()
        estadisticas=my_stast_filter.copy()
    else:
        tradesprev = pd.concat([tradesprev, tradesdf],ignore_index=True)
        estadisticas = pd.concat([estadisticas, my_stast_filter],ignore_index=True)
    # Mostramos
    print(f"Resultados para {ticker}:")
    print(stats)   
    #plt.title(f"Backtest -  {ticker}")
    bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")

Backtest.run:   0%|          | 0/2864 [00:00<?, ?bar/s]

Resultados para SPY:
<module 'scipy.stats' from 'C:\\Users\\carlo\\anaconda3\\Lib\\site-packages\\scipy\\stats\\__init__.py'>


C:\Users\carlo\AppData\Local\Temp\ipykernel_30924\1376799593.py:27: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")


Backtest.run:   0%|          | 0/2746 [00:00<?, ?bar/s]

Resultados para META:
<module 'scipy.stats' from 'C:\\Users\\carlo\\anaconda3\\Lib\\site-packages\\scipy\\stats\\__init__.py'>


C:\Users\carlo\AppData\Local\Temp\ipykernel_30924\1376799593.py:27: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")


Backtest.run:   0%|          | 0/2413 [00:00<?, ?bar/s]

C:\Users\carlo\anaconda3\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=1343: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


Resultados para AAPL:
<module 'scipy.stats' from 'C:\\Users\\carlo\\anaconda3\\Lib\\site-packages\\scipy\\stats\\__init__.py'>


C:\Users\carlo\AppData\Local\Temp\ipykernel_30924\1376799593.py:27: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")
